In [1]:
import warnings, os
warnings.filterwarnings("ignore")

from copy import copy
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score, matthews_corrcoef, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

import seaborn as sns
import matplotlib.pyplot as plt
from joblib import dump, load

from aggmap import AggMap, loadmap
from aggmap import AggMapNet as AggModel

from aggmap.AggMapNet import load_model, save_model
from aggmap import show


np.random.seed(666) #just for reaptable results

def tune_RF_params(X_train, y_train):
    n_estimators = [5, 20, 50, 100,200, 300] # number of trees in the random forest
    max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
    min_samples_split = [2, 6, 10] # minimum sample number to split a node
    min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
    max_features = ['auto', 'sqrt'] # number of features in consideration at every split
    bootstrap = [True, False] # method used to sample data points

    random_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'max_features':max_features,
                   'bootstrap':bootstrap,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}

    rf = RandomForestClassifier()
    rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid, 
                   n_iter = 100, cv = 5, verbose=0, random_state=35, n_jobs = -1)

    rf_random.fit(X_train, y_train)

    best_params = rf_random.best_params_

    return best_params


def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_pred

    '''
    the metrics are taken from orignal paper:
    Meta-Signer: Metagenomic Signature Identifier based on Rank Aggregation of Features
    https://github.com/YDaiLab/Meta-Signer/blob/bd6a1cd98d1035f848ecb6e53d9ee67a85871db2/src/utils/metasigner_io.py#L34
    '''
    auc = roc_auc_score(y_true, y_score, average='weighted')        
    mcc = matthews_corrcoef(y_true, y_pred)
    pres = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print('roc-auc: %.3f, mcc: %.3f, pres: %.3f, recall: %.3f, f1: %.3f' % (auc, mcc, pres, recall, f1))

    return auc, mcc, pres, recall, f1

2022-09-20 13:14:14.384081: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# read data

In [2]:
task = 'T2D'
data_path = '../../01_data/species_level/%s/' % (task)
save_dir = 'RF_results_%s' % task
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

dfa = pd.read_csv(os.path.join(data_path, 'abundance.tsv'),sep='\t', header=None, index_col=0)
dfb = pd.read_csv(os.path.join(data_path, 'labels.txt'),sep='\t', header=None)

dfx = dfa.T
dfy = dfb[0].map({'t2d':1, 'n':0})
Y = dfy.values

## use the same data process method
js = pd.read_json('../AggMapNet/T2D_results/HPs.json', orient = 'index')
fill = js.loc['featHPs'][0]['best_fill']
scale_method = js.loc['featHPs'][0]['best_scale_method']

dfx_new = pd.DataFrame(np.log(dfx + fill))
mp = AggMap(dfx_new)
mp = mp.fit(verbose = 0, var_thr = 0) 

X_3D = mp.batch_transform(dfx_new.values, scale_method=scale_method)

2022-09-20 13:14:15,114 - INFO - [bidd-aggmap] - Calculating distance ...
2022-09-20 13:14:15,124 - INFO - [bidd-aggmap] - the number of process is 16


100%|####################################################################################################################################| 606/606 [00:00<00:00, 4690.79it/s]


2022-09-20 13:14:22,503 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2022-09-20 13:14:25,222 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2022-09-20 13:14:25,667 - INFO - [bidd-aggmap] - Finished


100%|#####################################################################################################################################| 440/440 [00:02<00:00, 184.97it/s]


In [3]:
dfx_final  = mp.transform_mpX_to_df(X_3D)[mp.flist]
X = dfx_final.values
X.shape

(440, 606)

# 10FCV

In [4]:
outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128, 256, 1024, 2048, 4096, 8192] #10 repeats random seeds 8, 16, 32, 64, 128

each_fold_results = []
run_all_res = []

for i, repeat_seed in enumerate(repeat_seeds): 
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(dfy)), dfy)
    run_one_res = []
    for j, idx in enumerate(outer_idx):
        fold_num = "fold_%s" % str(j).zfill(2) 
        print('#'*50 + ' repeat_seed: %s; %s ' % (repeat_seed, fold_num) + '#'*50 )
        train_idx, test_idx = idx
        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]
        
        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        ## get best parameters 
        if (i == 0) & (j == 0):
            best_params = tune_RF_params(trainX, trainY)
            print(best_params)
            pd.Series(best_params).to_json(os.path.join(save_dir, 'HPs.json'))
            
        clf = RandomForestClassifier(**best_params, random_state = 666)
        clf.fit(trainX, trainY)  #, 


        pred_proba = clf.predict_proba(testX)
        y_true = testY
        y_score = pred_proba[:,1]
        y_pred = np.argmax(pred_proba, axis=1)
        
        dfr = pd.DataFrame([y_true, y_score, y_pred]).T
        dfr.columns = ['y_true', 'y_score', 'y_pred']
        dfr.index = dfy.iloc[test_idx].index
        auc, mcc, pres, recall, f1  = score(dfr)
        run_one_res.append(dfr)
        ts = pd.Series([auc, mcc, pres, recall, f1, i, repeat_seed]).round(3)
        ts.index = ['auc', 'mcc', 'pres', 'recall', 'f1', 'i', 'repeat_seed']
        print(ts.to_dict())
        each_fold_results.append(ts.to_dict())
    run_all_res.append(pd.concat(run_one_res))

################################################## repeat_seed: 8; fold_00 ##################################################

 input train and test X shape is (396, 606), (44, 606) 
{'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 30, 'bootstrap': False}
roc-auc: 0.715, mcc: 0.365, pres: 0.683, recall: 0.682, f1: 0.681
{'auc': 0.715, 'mcc': 0.365, 'pres': 0.683, 'recall': 0.682, 'f1': 0.681, 'i': 0.0, 'repeat_seed': 8.0}
################################################## repeat_seed: 8; fold_01 ##################################################

 input train and test X shape is (396, 606), (44, 606) 
roc-auc: 0.723, mcc: 0.327, pres: 0.668, recall: 0.659, f1: 0.655
{'auc': 0.723, 'mcc': 0.327, 'pres': 0.668, 'recall': 0.659, 'f1': 0.655, 'i': 0.0, 'repeat_seed': 8.0}
################################################## repeat_seed: 8; fold_02 ##################################################

 input train and test X shape is (39

In [5]:
pd.DataFrame(each_fold_results).groupby('repeat_seed').mean().mean()

auc       0.73442
mcc       0.32852
pres      0.66591
recall    0.66299
f1        0.66147
i         4.50000
dtype: float64

In [6]:
pd.DataFrame(each_fold_results).groupby('repeat_seed').std().mean()

auc       0.076977
mcc       0.138860
pres      0.069662
recall    0.069325
f1        0.069655
i         0.000000
dtype: float64

In [7]:
pd.DataFrame(each_fold_results).to_csv(os.path.join(save_dir, 'performance_results.csv'))